#### Prediction using the direct pickle file

In [1]:
import pickle
with open('model.pkl','rb') as f:
    model = pickle.load(f)

In [2]:
# Predicting using the loaded pickle file
single_prediction  = model.predict([[7.5, 0.65, 0.12, 2.1, 0.085, 12.0, 45.0, 0.9975, 3.35, 0.60, 10.2]])
single_prediction

/Users/ravikantkumar/anaconda3/envs/test_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])

#### Predicting from the pickle file which have been saved through mlflow

In [3]:
import mlflow

# This is the model which have been loaded
logged_model = 'runs:/a5896db75c1a4fea804ddcd5eb41a1dd/Classification-Model'

# Now, logged model needs to be loaded as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# NOw, prediciting with loaded model

single_prediction  = loaded_model.predict([[7.5, 0.65, 0.12, 2.1, 0.085, 12.0, 45.0, 0.9975, 3.35, 0.60, 10.2]])
single_prediction

/Users/ravikantkumar/anaconda3/envs/test_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array([1])

#### Loading the model from best run for a given experiment

In [6]:
# Importing the Mlflowclinet library
import mlflow
from mlflow.tracking import MlflowClient

In [7]:
# Intializing the MlflowClient to interact with the tracking server i.e. the records that 
# is being recorded in mlrun folder
client = MlflowClient()

In [9]:
# Defining the experiment name for which we want to fetch the details
experiment_name = "Wine Quality classification Experiment"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

In [13]:
experiment_id

'179020510512775425'

Order_by values are the values which are columns of the table for a given experiment_id or name.

In [72]:
# Getting all the runs for the given experiment_id
# https://mlflow.org/docs/latest/search-runs.html

runs = client.search_runs(
    experiment_ids = [experiment_id],
    # filter_strings = "", # Add filter string if needed
    # order_by  = ["metrics.accuracy Desc"] # One way to sort the runs
    order_by= ['Created desc'] # Another way to sort the runs
)

In [73]:
type(runs)

mlflow.store.entities.paged_list.PagedList

In [74]:
# This will give the best run as we have order the runs on the basis of accuracy
best_run = runs[0]

In [75]:
print(type(best_run))
print(best_run.info)
print()
print(best_run.info.run_id)

<class 'mlflow.entities.run.Run'>
<RunInfo: artifact_uri='/Users/ravikantkumar/myspace/model_testing/mlruns/179020510512775425/ebaa8106e4124e93aecf6b3879528c06/artifacts', end_time=1716043946365, experiment_id='179020510512775425', lifecycle_stage='active', run_id='ebaa8106e4124e93aecf6b3879528c06', run_name='fun-mole-236', run_uuid='ebaa8106e4124e93aecf6b3879528c06', start_time=1716043945217, status='FINISHED', user_id='ravikantkumar'>

ebaa8106e4124e93aecf6b3879528c06


In [76]:
best_run

<Run: data=<RunData: metrics={'Accuracy': 0.9125, 'Precision': 0.9125, 'Recall': 0.569078947368421}, params={}, tags={'Author': 'Rkk',
 'Version_run': '4',
 'mlflow.log-model.history': '[{"run_id": "ebaa8106e4124e93aecf6b3879528c06", '
                             '"artifact_path": "Classification-Model", '
                             '"utc_time_created": "2024-05-18 '
                             '14:52:25.221868", "flavors": {"python_function": '
                             '{"model_path": "model.pkl", "predict_fn": '
                             '"predict", "loader_module": "mlflow.sklearn", '
                             '"python_version": "3.12.3", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"sklearn": {"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.2", '
                             '"serialization_format": "cloudpickle", "code": '
                       

In [49]:
# As we can see that our model data is present in the mlflow.log-model.history. so we will
# get its details using the below code
best_run.data.tags.get('mlflow.log-model.history')

'[{"run_id": "ebaa8106e4124e93aecf6b3879528c06", "artifact_path": "Classification-Model", "utc_time_created": "2024-05-18 14:52:25.221868", "flavors": {"python_function": {"model_path": "model.pkl", "predict_fn": "predict", "loader_module": "mlflow.sklearn", "python_version": "3.12.3", "env": {"conda": "conda.yaml", "virtualenv": "python_env.yaml"}}, "sklearn": {"pickled_model": "model.pkl", "sklearn_version": "1.4.2", "serialization_format": "cloudpickle", "code": null}}, "model_uuid": "5394f48cd2314022b86b61dd23bb8eec", "mlflow_version": "2.12.2", "model_size_bytes": 15067}]'

In [61]:
type(best_run.data.tags.get('mlflow.log-model.history'))

str

In [62]:
# One thing we should remember that out that we are getting from best_run.data.tags.get('mlflow.log-model.history')
# is of str type as shown above but actually is of type json string.
# so will load the data in the json format
import json
json_str = best_run.data.tags.get('mlflow.log-model.history')
data  = json.loads(json_str)
data

[{'run_id': 'ebaa8106e4124e93aecf6b3879528c06',
  'artifact_path': 'Classification-Model',
  'utc_time_created': '2024-05-18 14:52:25.221868',
  'flavors': {'python_function': {'model_path': 'model.pkl',
    'predict_fn': 'predict',
    'loader_module': 'mlflow.sklearn',
    'python_version': '3.12.3',
    'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}},
   'sklearn': {'pickled_model': 'model.pkl',
    'sklearn_version': '1.4.2',
    'serialization_format': 'cloudpickle',
    'code': None}},
  'model_uuid': '5394f48cd2314022b86b61dd23bb8eec',
  'mlflow_version': '2.12.2',
  'model_size_bytes': 15067}]

In [63]:
# for getting the dictionary from it
data[0]

{'run_id': 'ebaa8106e4124e93aecf6b3879528c06',
 'artifact_path': 'Classification-Model',
 'utc_time_created': '2024-05-18 14:52:25.221868',
 'flavors': {'python_function': {'model_path': 'model.pkl',
   'predict_fn': 'predict',
   'loader_module': 'mlflow.sklearn',
   'python_version': '3.12.3',
   'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}},
  'sklearn': {'pickled_model': 'model.pkl',
   'sklearn_version': '1.4.2',
   'serialization_format': 'cloudpickle',
   'code': None}},
 'model_uuid': '5394f48cd2314022b86b61dd23bb8eec',
 'mlflow_version': '2.12.2',
 'model_size_bytes': 15067}

In [57]:
data[0]['run_id']

'ebaa8106e4124e93aecf6b3879528c06'

###### Below model_uri format 
**The model Uri format is same as we get it in the mlflow run-artifacts  as given below :**

import mlflow<br>
logged_model = 'runs:/ebaa8106e4124e93aecf6b3879528c06/Classification-Model'<br><br>

Load model as a PyFuncModel.<br>
loaded_model = mlflow.pyfunc.load_model(logged_model)<br><br>

Predict on a Pandas DataFrame.<br>
import pandas as pd<br>
loaded_model.predict(pd.DataFrame(data))<br>


In [59]:
# Loading the model from the best run
model_uri = f"runs:/{data[0]['run_id']}/{data[0]['artifact_path']}"
model = mlflow.sklearn.load_model(model_uri)


testing_value = [[7.5, 0.65, 0.12, 2.1, 0.085, 12.0, 45.0, 0.9975, 3.35, 0.60, 10.2]]

predictions = model.predict(testing_value)
print(predictions)

[1]


/Users/ravikantkumar/anaconda3/envs/test_env/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


##### IN case if wanted to get the all the values of the best_run then

In [64]:
# In case if we wanted to print all the details under the best run value then :
import json

# Provided JSON string
json_str = json_str

# Parse the JSON string
data = json.loads(json_str)

# Extract the dictionary (assuming there is only one dictionary in the list)
model_info = data[0]

# Print the values
print("run_id:", model_info["run_id"])
print("artifact_path:", model_info["artifact_path"])
print("utc_time_created:", model_info["utc_time_created"])
print("flavors:", model_info["flavors"])
print("model_uuid:", model_info["model_uuid"])
print("mlflow_version:", model_info["mlflow_version"])
print("model_size_bytes:", model_info["model_size_bytes"])

# If you want to access individual flavor details:
python_function_flavor = model_info["flavors"]["python_function"]
sklearn_flavor = model_info["flavors"]["sklearn"]

print("Python function flavor details:")
print("  model_path:", python_function_flavor["model_path"])
print("  predict_fn:", python_function_flavor["predict_fn"])
print("  loader_module:", python_function_flavor["loader_module"])
print("  python_version:", python_function_flavor["python_version"])
print("  env:", python_function_flavor["env"])

print("Sklearn flavor details:")
print("  pickled_model:", sklearn_flavor["pickled_model"])
print("  sklearn_version:", sklearn_flavor["sklearn_version"])
print("  serialization_format:", sklearn_flavor["serialization_format"])


run_id: ebaa8106e4124e93aecf6b3879528c06
artifact_path: Classification-Model
utc_time_created: 2024-05-18 14:52:25.221868
flavors: {'python_function': {'model_path': 'model.pkl', 'predict_fn': 'predict', 'loader_module': 'mlflow.sklearn', 'python_version': '3.12.3', 'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}}, 'sklearn': {'pickled_model': 'model.pkl', 'sklearn_version': '1.4.2', 'serialization_format': 'cloudpickle', 'code': None}}
model_uuid: 5394f48cd2314022b86b61dd23bb8eec
mlflow_version: 2.12.2
model_size_bytes: 15067
Python function flavor details:
  model_path: model.pkl
  predict_fn: predict
  loader_module: mlflow.sklearn
  python_version: 3.12.3
  env: {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}
Sklearn flavor details:
  pickled_model: model.pkl
  sklearn_version: 1.4.2
  serialization_format: cloudpickle
